In [1]:
import torch
import torch.nn as nn 
import math
from torch import nn, Tensor
import uproot
import numpy as np
import awkward as ak
from torch.utils.data import DataLoader
import torch
import datetime
import numpy as np
import torch
import torch.nn as nn 
import math
from torch import nn, Tensor
from torch.utils.data import Dataset
import torch.nn as nn 
from torch import nn, Tensor
import torch.nn.functional as F
import tqdm

In [127]:
import uproot
### Energy = 0.15 GeV

root_file1 = uproot.open(f"/eos/user/d/dasgupsu/SWAN_projects/ECAL_noise_EM_discrimination/data/outputPSWithPU_withNoise_0.150000_0.15_19Nov_VF1.root")
root_file2 = uproot.open(f"/eos/user/d/dasgupsu/SWAN_projects/ECAL_noise_EM_discrimination/data/outputPSWithPU_withNoise_0.150000_0.15_19Nov_VF2.root")
tree1 = root_file1["Samples"]
tree2 = root_file2["Samples"]
arrays1 = tree1.arrays(["samples", "ysamples","samplesNoise","ysamplesNoise", "waveform", "PUOnlywaveform"])
arrays2 = tree2.arrays(["samples", "ysamples","samplesNoise","ysamplesNoise", "PUOnlywaveform"])
X_real = ak.to_numpy(arrays1["samples"])
X_real = X_real/np.max(X_real, axis = 1).reshape(200000,1)
y_real = ak.to_numpy(arrays1["ysamples"])
X_noise = ak.to_numpy(arrays2["samplesNoise"])
X_noise = X_noise/np.max(X_noise, axis = 1).reshape(200000,1)
y_noise = ak.to_numpy(arrays2["ysamplesNoise"])
X_waveform = ak.to_numpy(arrays1["waveform"])
X_waveform = X_waveform / np.max(X_waveform, axis = 1).reshape(200000,1)
X_Zero = np.zeros((X_waveform.shape[0], X_waveform.shape[1]), dtype=float)

print(X_real) 
print("Printing y_real")
print(y_real)

data = np.concatenate([X_real, X_noise]) ### makes it [2*num_events,num_samples]
#data = X_real
labels = np.concatenate([y_real,y_noise])
#labels = y_real
X_target = np.concatenate([X_waveform, X_Zero])

shuffle_indices = np.random.permutation(len(data))
data = data[shuffle_indices]
labels = labels[shuffle_indices]
X_target = X_target[shuffle_indices]

[[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]]
Printing y_real
[[1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]
 [1.]]


In [128]:
import awkward as ak

num_events_data = ak.num(data, axis=0)
print(f'number of rows in data is {num_events_data}')


ntimeSamples_data = ak.num(data, axis=1)
print(f'number of rows in data is {ntimeSamples_data[0]}') ## just take the 0th event

# Split into train and test sets
train_size = int(0.5                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           * num_events_data) ###times 2 because the noise is also in the same dataset, so it is 2*num_events
valid_size = train_size + int(0.1 * num_events_data)
valid_size_down = train_size - int(0.1 * num_events_data)
train_data = data[:train_size]
train_labels = labels[:train_size]
train_target = X_target[:train_size]

valid_data = data[valid_size_down:valid_size]
valid_labels = labels[valid_size_down:valid_size]
valid_target = X_target[valid_size_down:valid_size]

test_data = data[valid_size:]
test_labels = labels[valid_size:]
test_target = X_target[valid_size:]

print(f'Size of training data is {train_size}')

'''
print(train_size)
print(train_data)
print(test_data)
'''

print(f'number of elements in data : training data : test data : validation data: {len(data)} : {len(train_data)} : {len(test_data)} : {len(valid_data)}')

number of rows in data is 20
number of rows in data is 10
Size of training data is 10
number of elements in data : training data : test data : validation data: 20 : 10 : 8 : 4


In [129]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print("device:", device)

device: cuda


In [130]:
train_data = train_data.reshape((train_data.shape[0], train_data.shape[1], 1))
test_data = test_data.reshape((test_data.shape[0], test_data.shape[1], 1))
valid_data = valid_data.reshape((valid_data.shape[0], valid_data.shape[1], 1))
train_labels = train_labels.reshape((train_labels.shape[0], 1))
test_labels = test_labels.reshape((test_labels.shape[0], 1))
valid_labels = valid_labels.reshape((valid_labels.shape[0], 1))
train_target = train_target.reshape((train_target.shape[0], train_target.shape[1],1))
test_target = test_target.reshape((test_target.shape[0], test_target.shape[1],1))
valid_target = valid_target.reshape((valid_target.shape[0], valid_target.shape[1],1))

In [131]:
train_data = torch.tensor(train_data, dtype =torch.float).to(device)
test_data = torch.tensor(test_data, dtype =torch.float).to(device)
valid_data = torch.tensor(valid_data, dtype =torch.float).to(device)
train_labels = torch.tensor(train_labels, dtype =torch.float).to(device)
test_labels = torch.tensor(test_labels, dtype =torch.float).to(device)
valid_labels = torch.tensor(valid_labels, dtype =torch.float).to(device)
train_target = torch.tensor(train_target, dtype =torch.float).to(device)
test_target = torch.tensor(test_target, dtype =torch.float).to(device)
valid_target = torch.tensor(valid_target, dtype =torch.float).to(device)

In [132]:
class TransformerDataset(Dataset):
    """
    Dataset class used for transformer models.
    
    """
    def __init__(self, 
        data: torch.tensor,
        target: torch.tensor,
        device
        ) -> None:
        
        super().__init__()

        self.data = data
        self.target = target
        self.device = device

        print("data size = {}".format(data.size()))
        print("target size = {}".format(target.size()))
            

    def __len__(self):
        
        return self.data.size()[0]

    def __getitem__(self, index):
        """
        Returns a tuple with 3 elements:
        1) src (the encoder input)
        2) trg (the decoder input)
        3) trg_y (the target)
        """
        #print(self.data.size())
        src = self.data[index]
        
        start_token = 99.*torch.ones((self.target.size()[0],1,1)).to(self.device)
        target = torch.cat((start_token, self.target),1)[index]
        
        trg = target[:-1,:]
        trg_y = target[1:,:]
        

        return src, trg, trg_y

In [133]:
training_data = TransformerDataset(
    data = train_data,
    target = train_target,
    device = device)

data size = torch.Size([10, 10, 1])
target size = torch.Size([10, 10, 1])


In [134]:
testing_data = TransformerDataset(
    data = test_data,
    target = test_target,
    device = device)

data size = torch.Size([8, 10, 1])
target size = torch.Size([8, 10, 1])


In [135]:
validing_data = TransformerDataset(
    data = valid_data,
    target = valid_target,
    device = device)

data size = torch.Size([4, 10, 1])
target size = torch.Size([4, 10, 1])


In [136]:
batch_size=10
# Load the training data
training_dataloader = DataLoader(training_data, batch_size= batch_size, drop_last = True)
# Load the testing data
testing_dataloader = DataLoader(testing_data, batch_size= batch_size, drop_last = True)
# Load the validing data
validing_dataloader = DataLoader(validing_data, batch_size= batch_size, drop_last = True)

In [137]:
class Encoder(nn.Module):
    def __init__(self, input_size, embedding_dim, hidden_dim, n_layers, dropout_prob):
        super().__init__()
        #self.embedding = nn.Linear(
        #    in_features=input_size, 
        #    out_features=embedding_dim 
        #    )
        #self.activation = nn.Sigmoid()
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, bidirectional= True, dropout=dropout_prob, batch_first= True)
        self.dropout = nn.Dropout(dropout_prob)
 
    def forward(self, src):
        #print("Encoder input size:", src.size())
        #embed = self.dropout(self.embedding(src))
        #embed = self.activation(embed)
        embed = src
        #print("Encoder embedding input size:", embed.size())
        outputs, (hidden, cell) = self.rnn(embed)
        #print("Encoder hidden and cell", hidden.size(), cell.size())
        return hidden, cell

In [138]:
class OneStepDecoder(nn.Module):
    def __init__(self, input_size, embedding_dim, hidden_dim, n_layers, dropout_prob):
        super().__init__()
        self.input_size = input_size
        #self.embedding = nn.Linear(
            #in_features=input_size, 
            #out_features=embedding_dim 
            #)
        #self.activation1 = nn.Sigmoid()
        self.rnn = nn.LSTM(embedding_dim, hidden_dim, n_layers, bidirectional= True, dropout=dropout_prob)
        self.fc = nn.Linear(2*hidden_dim, input_size)
        self.activation2  = nn.Tanh()
        self.dropout = nn.Dropout(dropout_prob)
 
    def forward(self, target_token, hidden, cell):
        target_token = target_token.unsqueeze(0)
        #print("Target token:", target_token.size())
        #embedding_layer = self.dropout(self.embedding(target_token))
        #embedding_layer = self.activation1(embedding_layer)
        embedding_layer = target_token
        #print("Decoder Input dim", target_token.size(), hidden.size(), cell.size(), embedding_layer.size())
        output, (hidden, cell) = self.rnn(embedding_layer, (hidden, cell))
        #linear = self.fc(output)
        #print("Decoder Output dim", linear.size(), hidden.size(), cell.size())
        linear = self.fc(output.squeeze(0))
        linear = self.activation2(linear)
        #print("Decoder Output dim after squeeze",linear.size())
        return linear, hidden, cell

In [139]:
class Decoder(nn.Module):
    def __init__(self, one_step_decoder, device):
        super().__init__()
        self.one_step_decoder = one_step_decoder
        self.device=device
 
    def forward(self, target, hidden, cell):
        target_len, batch_size = target.shape[1], target.shape[0]
        target_vocab_size = self.one_step_decoder.input_size
        predictions = torch.zeros(target_len, batch_size, target_vocab_size).to(self.device)
        input = target[:,0,:]
        for t in range(1, target_len):
            predict, hidden, cell = self.one_step_decoder(input, hidden, cell)
            index = t-1
            predictions[index] = predict            
            input= target[:,t,:]            
        return predictions

In [140]:
class EncoderDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder        
 
    def forward(self, source, target):        
        hidden, cell = self.encoder(source)
        outputs = self.decoder(target, hidden, cell)
        return outputs

In [141]:
def create_model():
    # Define the required dimensions and hyper parameters
    input_size = 1
    embedding_dim = 1
    hidden_dim = 32
    dropout = 0.0
 
    # Instanciate the models
    encoder = Encoder(input_size, embedding_dim, hidden_dim, n_layers=1, dropout_prob=dropout)
    one_step_decoder = OneStepDecoder(input_size, embedding_dim, hidden_dim, n_layers=1, dropout_prob=dropout)
    decoder = Decoder(one_step_decoder, device)
 
    model = EncoderDecoder(encoder, decoder)
    model = model.to(device)
 
    # Define the optimizer
    optimizer = torch.optim.Adam(model.parameters())
    criterion = nn.MSELoss()
 
    return model, optimizer, criterion

In [144]:
def train(training_dataloader, validing_dataloader, epochs = 10):
    
    model, optimizer, criterion = create_model()
    
    for epoch in range(1, epochs + 1):
        training_loss = []
        model.train()
        
        for i, (src, trg, trg_y) in enumerate(training_dataloader):

            optimizer.zero_grad()
                
            output = model(src, trg)
            
            output = output.reshape(output.shape[1], output.shape[0], output.shape[2])

            loss = criterion(output, trg_y)
                     
            loss.backward()
            
            optimizer.step()
 
            training_loss.append(loss.item())
        
        print(f"At epoch {epoch} mean training loss is {torch.mean(torch.FloatTensor(training_loss))}")
        print(f"Prediction: {output[15]}, Target: {trg_y[15]}")
        
        with torch.no_grad():
        
            model.eval()
            
            validation_loss = []
            
            for i, (src, trg, trg_y) in enumerate(validing_dataloader):
            
                output = model(src, trg)
                output = output.reshape(output.shape[1], output.shape[0], output.shape[2])
                
                loss = criterion(output, trg_y)
                
                validation_loss.append(loss.item())
            
            print(f"At epoch {epoch} mean validation loss is {torch.mean(torch.FloatTensor(validation_loss))}")
            print(f"Prediction: {output[15]}, Target: {trg_y[15]}")
            
 
    return model

In [145]:
model = train(training_dataloader, validing_dataloader, epochs = 2)

src: tensor([[[-0.1189],
         [-0.1032],
         [ 0.0081],
         [ 0.2161],
         [ 0.6688],
         [ 0.9865],
         [ 1.0000],
         [ 0.9008],
         [ 0.7529],
         [ 0.3929]],

        [[ 0.7294],
         [ 0.7887],
         [ 0.7947],
         [ 0.8111],
         [ 1.0000],
         [ 0.9595],
         [ 0.9038],
         [ 0.6035],
         [ 0.5672],
         [ 0.4559]],

        [[ 0.3449],
         [-0.0873],
         [ 0.0666],
         [ 0.3518],
         [ 0.6742],
         [ 0.7206],
         [ 1.0000],
         [ 0.7730],
         [ 0.0455],
         [ 0.0031]],

        [[ 0.8006],
         [ 0.8397],
         [ 1.0000],
         [ 0.7899],
         [ 0.3102],
         [ 0.0546],
         [-0.1327],
         [-0.1938],
         [-0.1921],
         [-0.1413]],

        [[ 0.3956],
         [ 0.4032],
         [ 0.4028],
         [ 0.0993],
         [ 0.8090],
         [ 0.9610],
         [ 0.3144],
         [-0.2470],
         [-0.6988],
       

input: tensor([[1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.]], device='cuda:0')
1: tensor([[[1.],
         [1.],
         [1.],
         [0.],
         [1.],
         [1.],
         [1.],
         [0.],
         [1.],
         [0.]]], device='cuda:0')
2: tensor([[[1.],
         [1.],
         [1.],
         [0.],
         [1.],
         [1.],
         [1.],
         [0.],
         [1.],
         [0.]]], device='cuda:0')
3: tensor([[[ 0.0026, -0.0033, -0.1195,  0.0099, -0.0189,  0.0723, -0.0865,
          -0.1063, -0.1127,  0.1501,  0.0719, -0.0195, -0.0518, -0.0655,
          -0.0238, -0.0942, -0.0500,  0.0169,  0.0910, -0.0434,  0.0768,
           0.0923,  0.0321,  0.0937, -0.0737, -0.1623, -0.0886,  0.0674,
          -0.0085, -0.0124, -0.0329,  0.0690, -0.1201, -0.0477, -0.0104,
          -0.0628,  0.0215,  0.0606, -0.1718, -0.0692, -0.1065, -0.0610,
           0.0555, -0.1557,  0.0663,  0.0588,  0.0

input: tensor([[0.1762],
        [0.1014],
        [0.0390],
        [0.0000],
        [0.1811],
        [0.6997],
        [0.1518],
        [0.0000],
        [0.1614],
        [0.0000]], device='cuda:0')
1: tensor([[[0.1762],
         [0.1014],
         [0.0390],
         [0.0000],
         [0.1811],
         [0.6997],
         [0.1518],
         [0.0000],
         [0.1614],
         [0.0000]]], device='cuda:0')
2: tensor([[[0.1762],
         [0.1014],
         [0.0390],
         [0.0000],
         [0.1811],
         [0.6997],
         [0.1518],
         [0.0000],
         [0.1614],
         [0.0000]]], device='cuda:0')
3: tensor([[[ 0.0399,  0.0119, -0.1173, -0.0516,  0.0161,  0.0798, -0.0236,
          -0.0939, -0.0886,  0.1767,  0.0585,  0.0463, -0.1070, -0.0834,
          -0.0425, -0.1511,  0.0355,  0.0326,  0.0654,  0.0005,  0.1619,
           0.0977, -0.0142,  0.1285, -0.1193, -0.1291, -0.1095,  0.1344,
           0.0302,  0.0588, -0.0801,  0.0639, -0.1347, -0.0721, -0.0443,
   

In [19]:
model

EncoderDecoder(
  (encoder): Encoder(
    (embedding): Linear(in_features=1, out_features=256, bias=True)
    (activation): Sigmoid()
    (rnn): LSTM(256, 512, num_layers=2, batch_first=True, bidirectional=True)
    (dropout): Dropout(p=0.0, inplace=False)
  )
  (decoder): Decoder(
    (one_step_decoder): OneStepDecoder(
      (embedding): Linear(in_features=1, out_features=256, bias=True)
      (activation1): Sigmoid()
      (rnn): LSTM(256, 512, num_layers=2, bidirectional=True)
      (fc): Linear(in_features=1024, out_features=1, bias=True)
      (activation2): Tanh()
      (dropout): Dropout(p=0.0, inplace=False)
    )
  )
)